In [ ]:
# import all necessary libraries
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt

In [ ]:
# read all csv files
all_course_paid_bankin = pd.read_csv('all_course_paid_bankin.csv')
free_reg = pd.read_csv('free_reg.csv', keep_default_na=False)
paid = pd.read_csv('paid.csv')
paid_cred = pd.read_csv('paid_cred.csv')
activecam_sub = pd.read_csv('activecam_sub.csv')

In [ ]:
free_reg_after=free_reg

In [ ]:
activecam_sub1= activecam_sub[activecam_sub["Date Created"] >= "2021-07-13"]

In [ ]:
# Since all_course_paid_bankin contains all courses, we need to extract the Capital Reduction course from all_course_paid_bankin and save it as paid_bankin
paid_bankin = all_course_paid_bankin[all_course_paid_bankin['Course Name'].str.contains("capital", case = False, na = False)]
# Extract ActiveCampaign subscribers that subscribed before 13/7
activecam_sub = activecam_sub[activecam_sub["Date Created"] < "2021-07-13"]

In [ ]:
# Refer to Trinly, extract all columns needed
paid_bankin = paid_bankin[["Timestamp", "Email Address", "Your Name (Put in Learnabee)", "Your Whatsapp Number"]]
free_reg = free_reg[["First name", "Last name", "Phone number", "Email", "Registration date", "Attended live"]]
paid = paid[["fullname", "email"]]
paid_cred = paid_cred[["purchased_at", "user", "user_email"]]
activecam_sub = activecam_sub[["Email"]]

In [ ]:
#Data Cleansing - Make email address lower case
free_reg["Email"] = free_reg["Email"].str.lower()
free_reg
# Data Cleansing - Join First name and Last name into Name, then capitalise first letters of both
free_reg["Name"] = free_reg["First name"] + " " + free_reg["Last name"]
free_reg["Name"] = free_reg['Name'].str.title()
# rearrange collumns to keep 
free_reg1 = free_reg[["Name", "Phone number", "Email", "Registration date", "Attended live"]]
free_reg1.head()

In [ ]:
# Data Cleansing - Make email address lower case
activecam_sub["Email"] = activecam_sub["Email"].str.lower()
activecam_sub1["Email"]=activecam_sub1["Email"].str.lower()
# Remove duplicates
activecam_sub = activecam_sub.drop_duplicates(subset = "Email")
activecam_sub1 = activecam_sub1.drop_duplicates(subset = "Email")

In [ ]:
#DC-finding duplicates and view them to analyse which to keep
duplicates = pd.concat(g for _, g in free_reg1.groupby("Email") if len(g) > 1)
duplicates1 = duplicates[["Name", "Email", "Attended live"]]
print(duplicates1)
#since 2nd duplicate indicates yes or no, remove first duplicate

In [ ]:
#DC-drop duplicates and keep last entry only
free_reg2 = free_reg1.drop_duplicates(subset="Email", keep="last")  
#Remove entries with duplicated phone numbers
free_reg2 = free_reg2.drop_duplicates(subset="Phone number", keep="last")
dup_phone = free_reg2[free_reg2.duplicated(['Phone number'])]
print("Duplicated rows based on phone numbers are:", dup_phone, sep='\n')

In [ ]:
# Change the data type of phone number to string
free_reg3 = free_reg2["Phone number"].astype(str)
# Reformatting the phone number
for i in range(len(free_reg3)):
  if free_reg3.iloc[i][0] == '6':
    free_reg3.iloc[i] = "+" + free_reg3.iloc[i]
  elif len(free_reg3.iloc[i]) < 9:
    free_reg3.iloc[i] = "+603" + free_reg3.iloc[i]
  elif (len(free_reg3.iloc[i]) >= 9) and (free_reg3.iloc[i][0] == '1' or free_reg3.iloc[i][0] == '3'):
    free_reg3.iloc[i] = "+60" + free_reg3.iloc[i]
  else:
    free_reg3.iloc[i] = "+6" + free_reg3.iloc[i]
free_reg2["Phone number"] = free_reg3

In [ ]:
# Category 4 - non-paid users in free_reg but not in ActiveCampaign list attended live
np_category4 = free_reg2[~free_reg2["Email"].isin(paid.email)]
np_category4 = np_category4[~np_category4["Email"].isin(activecam_sub.Email)]
np_category4 = np_category4[np_category4['Attended live'].str.contains("Yes")]
len(np_category4)

In [ ]:
# Category 5 - non-paid users in free_reg but not in ActiveCampaign list did not attend live
np_category5 = free_reg2[~free_reg2["Email"].isin(paid.email)]
np_category5 = np_category5[~np_category5["Email"].isin(activecam_sub.Email)]
np_category5 = np_category5[np_category5['Attended live'].str.contains("No")]
len(np_category5)

In [ ]:
# Category 1 - Registered = Yes, Attended live = Yes, Paid = No
np_category1 = free_reg2[free_reg2['Attended live'].str.contains("Yes")]
np_category1 = np_category1[np_category1["Email"].isin(activecam_sub.Email)] #trinly-prevoiusly wasnt finding for <13 july
np_category1 = np_category1[~np_category1["Email"].isin(paid.email)]
np_category1 = np_category1[["Name", "Phone number", "Email"]]
np_category1.reset_index(drop = True, inplace = True)
# Count Entries(no of registrants/rows)
len(np_category1.index)

In [ ]:
# Category 2 - Registered = Yes, Attended live = No, Paid = No
np_category2 = free_reg2[free_reg2['Attended live'].str.contains("No")]
np_category2 = np_category2[np_category2["Email"].isin(activecam_sub.Email)] #trinly-prevoiusly wasnt finding for <13 july
np_category2 = np_category2[~np_category2["Email"].isin(paid.email)]
np_category2 = np_category2[["Name", "Phone number", "Email"]]
np_category2.reset_index(drop = True, inplace = True)
# Count Entries- shows that dexters code succesfully narrowed down entries
len(np_category2.index)

In [ ]:
#added and cleaned at [19] part of code so can remove this??-Trinly
dup_phone = free_reg2[free_reg2.duplicated(['Phone number'])]
print("Duplicated rows based on phone numbers are:", dup_phone, sep='\n')

In [ ]:
# Category 3 - Registered = No, Attended live = No, Paid = No
np_category3 = free_reg2[free_reg2["Registration date"].isna()]
len(np_category3)

In [ ]:
# Export data 
#from google.colab import drive
#drive.mount('/drive', force_remount = True)
#np_category1.to_csv('/drive/My Drive/nonpaid_category1.csv')
#np_category2.to_csv('/drive/My Drive/nonpaid_category2.csv')

In [ ]:
# Data Cleansing - Make email address lower case
paid["email"] = paid["email"].str.lower()
paid_bankin["Email Address"] = paid_bankin["Email Address"].str.lower()
paid_cred["user_email"] = paid_cred["user_email"].str.lower()
# Remove duplicates
paid = paid.drop_duplicates(subset = "email")
paid_bankin = paid_bankin.drop_duplicates(subset = "Email Address")
paid_cred = paid_cred.drop_duplicates(subset = "user_email")
# Reformat name
paid["fullname"] = paid["fullname"].str.title()
paid_bankin["Your Name (Put in Learnabee)"] = paid_bankin["Your Name (Put in Learnabee)"].str.title()
paid_cred["user"] = paid_cred["user"].str.title()

In [ ]:
paid_after=paid

In [ ]:
# Length of each category
print(len(paid), len(paid_bankin), len(paid_cred))
# Conclusion: paid(254) = paid_bankin(25) + paid_cred(223) + paid_free(5)
# We want to identify paid_free

In [ ]:
# Identify paid_free
paid_free = paid[~paid["email"].isin(paid_bankin["Email Address"])]
paid_free = paid_free[~paid_free["email"].isin(paid_cred["user_email"])]
# However, len(paid_free) is 8 but not 6, so we filter out again by comparing name
paid_free = paid_free[~paid_free["fullname"].isin(paid_cred["user"])]
paid_free = paid_free[~paid_free["fullname"].isin(paid_bankin["Your Name (Put in Learnabee)"])]
# We can identify that their email is not consistent/ type wrongly, now len(paid_free) is 7
# remove Lynn Koay (special case), now the length is 6 as desired
paid_free = paid_free[paid_free["fullname"]!="Lynn Koay"]
print(len(paid_free))

In [ ]:
#Lookup and add available phone numbers to paid
#Change col name to be able to merge based on email
paid = paid.rename(columns={'email': 'Email'})

#merge to get phone numbers in paid master list
paid2= pd.merge(free_reg2,paid,how='right')
paid2= paid2[["fullname","Email","Phone number"]]
paid2

In [ ]:
#Lookup and add available phone numbers tp paid_cred
paid_cred = paid_cred.rename(columns={'user_email': 'Email'})
paid_cred2=pd.merge(paid_cred,free_reg2,how='left')
paid_cred2=paid_cred2[['purchased_at','user','Email','Phone number']]
paid_cred2.head()
#paid_bankin already has phone numbers

In [ ]:
# Clean phone numbers for paid_bankin
# Change the data type of phone number to string
paid_bankin2 = paid_bankin["Your Whatsapp Number"].astype(str)
# Reformatting the phone number
for i in range(len(paid_bankin2)):
  paid_bankin2.iloc[i] = paid_bankin2.iloc[i].replace(' ','')
  if '-' in paid_bankin2.iloc[i]:
    paid_bankin2.iloc[i] = paid_bankin2.iloc[i].replace('-','')
  if paid_bankin2.iloc[i][0] == '6':
    paid_bankin2.iloc[i] = "+" + paid_bankin2.iloc[i]
  elif len(paid_bankin2.iloc[i]) < 9:
    paid_bankin2.iloc[i] = "+603" + paid_bankin2.iloc[i]
  elif (len(paid_bankin2.iloc[i]) >= 9) and (paid_bankin2.iloc[i][0] == '1' or paid_bankin2.iloc[i][0] == '3'):
    paid_bankin2.iloc[i] = "+60" + paid_bankin2.iloc[i]
  else:
    paid_bankin2.iloc[i] = "+6" + paid_bankin2.iloc[i]
paid_bankin["Your Whatsapp Number"] = paid_bankin2
paid_bankin.reset_index(drop = True, inplace = True)

In [ ]:
from google.colab import drive
#drive.mount('/drive', force_remount = True)
#paid2.to_csv('/drive/My Drive/paid.csv')
#paid_bankin.to_csv('/drive/My Drive/paid_bankin.csv')
#paid_cred2.to_csv('/drive/My Drive/paid_cred.csv')
#paid_free.to_csv('/drive/My Drive/paid_free.csv')

In [ ]:
#Make all paid list where all_paid= paid_bankin + paid_cred
paid_bankin3= paid_bankin[["Your Name (Put in Learnabee)","Email Address","Your Whatsapp Number"]]
paid_bankin3=paid_bankin3.rename(columns={'Email Address':'Email'})
paid_bankin3= paid_bankin3.rename(columns={'Your Name (Put in Learnabee)':'Name'})
paid_bankin3= paid_bankin3.rename(columns={'Your Whatsapp Number':'Phone number'})
paid_cred3=paid_cred2[['user','Email','Phone number']]
paid_cred3= paid_cred3.rename(columns={'user':'Name'})
paid_all= pd.concat([paid_cred3,paid_bankin3],axis=0)
len(paid_all)

In [ ]:
# After indentified paid_free, find out adj_paid
paid_bankin = paid_bankin.rename(columns = {"Your Name (Put in Learnabee)": "name", "Email Address": "email", "Your Whatsapp Number": "phone_num"})
paid_cred2 = paid_cred2.rename(columns = {"user": "name", "Phone number": "phone_num", "Email": "email"})
adj_paid = pd.merge(paid_bankin, paid_cred2, how = "outer")

In [ ]:
adj_paid2= adj_paid[["Timestamp","name", "email", "phone_num"]]
adj_paid2.head()

In [ ]:
adj_paid = adj_paid[["name", "email", "phone_num"]]
len(adj_paid)

In [ ]:
#paid, registered and attended free session
p_category1= pd.merge(paid_all,free_reg2,on='Email',how='left')
p_category1=p_category1[p_category1["Email"].isin(activecam_sub.Email)]
p_category1=p_category1.loc[p_category1['Attended live'].isin(['Yes'])]
p_category1=p_category1[["Name_x","Email","Phone number_x","Registration date","Attended live"]]
p_category1.head()
len(p_category1)

In [ ]:
# Paid audience analysis
# Category 1 - Registered = Yes, Attended live = Yes
yes_live = free_reg2[free_reg2["Attended live"].str.contains("Yes")]
pp_category1 = adj_paid[adj_paid["email"].isin(yes_live.Email)]
pp_category1 = pp_category1[pp_category1["email"].isin(activecam_sub.Email)]
pp_category1.reset_index(drop = True, inplace = True)
len(pp_category1)

In [ ]:
#paid, registered and did not attend free session
p_category2= pd.merge(paid_all,free_reg2,on='Email',how='left')
p_category2=p_category2.loc[p_category2['Attended live'].isin(['No'])]
p_category2=p_category2[p_category2["Email"].isin(activecam_sub.Email)]
p_category2=p_category2[["Name_x","Email","Phone number_x","Registration date","Attended live"]]
p_category2.head()
len(p_category2)

In [ ]:
# Paid audience analysis
# Category 2 - Registered = Yes, Attended live = No
no_live = free_reg2[free_reg2["Attended live"].str.contains("No")]
pp_category2 = adj_paid[adj_paid["email"].isin(no_live.Email)]
pp_category2 = pp_category2[pp_category2["email"].isin(activecam_sub.Email)]
pp_category2.reset_index(drop = True, inplace = True)
len(pp_category2)

In [ ]:
p_category3=pd.merge(paid_all,free_reg2,on='Email',how='left')
p_category3=p_category3[p_category3['Registration date'].isna()]
p_category3=p_category3[p_category3["Email"].isin(activecam_sub.Email)]
p_category3=p_category3[["Name_x","Email","Phone number_x","Registration date","Attended live"]]
len(p_category3)
#If they didnt register they cannot attend the live then attended live is n/a which means they did not attend live

In [ ]:
# Paid audience analysis
# Category 3 - Registered = No, Attended live = No
pp_category3 = adj_paid[~adj_paid["email"].isin(free_reg2.Email)]
pp_category3 = pp_category3[pp_category3["email"].isin(activecam_sub.Email)]
pp_category3.reset_index(drop = True, inplace = True)
len(pp_category3)

In [ ]:
p_category4= pd.merge(paid_all,free_reg2,on='Email',how='left')
p_category4=p_category4[~p_category4["Email"].isin(activecam_sub.Email)]

p_category4=p_category4[["Name_x","Email","Phone number_x","Registration date","Attended live"]]
len(p_category4)
p_category4.head()

In [ ]:
# Paid audience analysis
# Category 4 - not in activecampaign, registered = yes, attended live = yes
pp_category4 = adj_paid[~adj_paid["email"].isin(activecam_sub.Email)]
pp_category4 = pp_category4[pp_category4["email"].isin(free_reg2.Email)]
pp_category4 = pp_category4[pp_category4["email"].isin(yes_live.Email)]
pp_category4.reset_index(drop = True, inplace = True)
len(pp_category4)

In [ ]:
# Category 5 - not in activecampaign, registered = yes, attended live = no
pp_category5 = adj_paid[~adj_paid["email"].isin(activecam_sub.Email)]
pp_category5 = pp_category5[pp_category5["email"].isin(free_reg2.Email)]
pp_category5 = pp_category5[pp_category5["email"].isin(no_live.Email)]
pp_category5.reset_index(drop = True, inplace = True)
len(pp_category5)

In [ ]:
# Category 6 - not in activecampaign, registered = no, attended live = no
pp_category6 = adj_paid[adj_paid["email"].isin(activecam_sub.Email)]
pp_category6 = pp_category6[~pp_category6["email"].isin(free_reg2.Email)]
pp_category6 = pp_category6[pp_category6["email"].isin(no_live.Email)]
pp_category6.reset_index(drop = True, inplace = True)
len(pp_category6)

In [ ]:
#drive.mount('/drive', force_remount = True)
#pp_category1.to_csv('/drive/My Drive/paid_category1.csv')
#pp_category2.to_csv('/drive/My Drive/paid_category2.csv')
#pp_category3.to_csv('/drive/My Drive/paid_category3.csv')
#pp_category4.to_csv('/drive/My Drive/paid_category4.csv')
#signup_trend2.to_csv('/drive/My Drive/signup_trend.csv')

In [ ]:
#for signup trend with drop down list
paid_cred4=paid_cred
paid_cred4['Category']="Credit_card"
paid_cred4=paid_cred4[["Category","purchased_at","Email"]]
paid_cred4=paid_cred4.rename(columns={"purchased_at":"date","Email":"email"})
paid_cred4.sort_values(by=['date'], inplace=True, ascending=True)
paid_cred4.head()

In [ ]:
#for signup trend with drop down list
paid_bankin4=paid_bankin
paid_bankin4['Category']="Bank_in"
paid_bankin4=paid_bankin4[["Category","Timestamp","email"]]
paid_bankin4=paid_bankin4.rename(columns={"Timestamp":"date"})
paid_bankin4.sort_values(by=['date'], inplace=True, ascending=True)
paid_bankin4.head()
len(paid_bankin4)

In [ ]:
#for signup trend with drop down list
signup_trend3=pd.concat([paid_cred4,paid_bankin4],axis=0)
signup_trend3.head()

In [ ]:
#new leads
#number of registrants that joined ON or AFTER 13 july
free_reg_after=free_reg_after[free_reg_after["Email"].isin(activecam_sub1.Email)]
print(len(free_reg_after))
free_reg_after=free_reg_after[~free_reg_after["Email"].isin(activecam_sub.Email)]
print(len(free_reg_after))

In [ ]:
# Sign-up trend analysis (paid_bankin and paid_cred2)
# All reformat into MM/DD
# paid_bankin current timestamp format: M/D/YYYY HH:MM:SS
# paid_cred2 current timestamp format: YYYY-MM-DD
paid_cred_time = paid_cred["purchased_at"]
paid_bankin_time = paid_bankin["Timestamp"]
for i in range(len(paid_cred_time)):
  d = datetime.datetime.strptime(paid_cred_time.iloc[i], "%Y-%m-%d")
  paid_cred_time.iloc[i] = datetime.date.strftime(d, "%m/%d")
for i in range(len(paid_bankin_time)):
  d = datetime.datetime.strptime(paid_bankin_time.iloc[i], "%m/%d/%Y %H:%M:%S")
  paid_bankin_time.iloc[i] = datetime.date.strftime(d, "%m/%d")
paid_cred_time = paid_cred_time.to_frame()
paid_cred_time = paid_cred_time.rename(columns = {"purchased_at": "date"})
paid_bankin_time = paid_bankin_time.to_frame()
paid_bankin_time = paid_bankin_time.rename(columns = {"Timestamp": "date"})

In [ ]:
# Group by date
paid_cred_time["credit_card"] = 1
paid_cred_time = paid_cred_time.groupby(["date"]).sum()
paid_bankin_time["bank_in"] = 1
paid_bankin_time = paid_bankin_time.groupby(["date"]).sum()

In [ ]:
# Merge them together
signup_trend = paid_cred_time.merge(paid_bankin_time, how = "left", on = "date")
signup_trend = signup_trend.fillna(0)
signup_trend.sort_values(by = "date")
signup_trend = signup_trend.astype(int)
signup_trend.reset_index(inplace = True)
# Change the date format into DD/MM
signup_trend['date'] = pd.to_datetime(signup_trend['date'], format = "%m/%d").dt.strftime("%d/%m")
signup_trend["total"] = signup_trend["credit_card"] + signup_trend["bank_in"]
signup_trend.head()

In [ ]:
# To draw a trend line, we should include all dates from 21/6 until 1/8 in signup_trend
alldate = pd.date_range(start="2021-06-21",end="2021-08-30").tolist()
alldate = pd.DataFrame(alldate, columns = ["date"])
alldate['date'] = pd.to_datetime(alldate['date'], format = "%Y-%m-%d").dt.strftime("%d/%m")
alldate.head()

In [ ]:
# Merge alldate and signup_trend
signup_trend2 = pd.merge(alldate, signup_trend, how = "left")
signup_trend2 = signup_trend2.fillna(0)
signup_trend2[["credit_card", "bank_in", "total"]] = signup_trend2[["credit_card", "bank_in", "total"]].astype(int)
signup_trend2.head()

In [ ]:
# signup_trend2 visualisation, suggest to use stacked bar chart
signup_trend2[["date", "credit_card", "bank_in"]].plot(x='date', kind='bar', stacked=True, title='Sign-up Trend Analysis', figsize = (12, 4))

In [ ]:
#signup trend for paid
pp1_trend=pd.merge(pp_category1,signup_trend3,how='left')
pp1_trend=pp1_trend[["date","name","email","phone_num","Category"]]
pp1_trend=pp1_trend.rename(columns={"Category":"Method"})
pp1_trend['category']="p1:AC1,R1,AL1"

pp2_trend=pd.merge(pp_category2,signup_trend3,how='left')
pp2_trend=pp2_trend[["date","name","email","phone_num","Category"]]
pp2_trend['category']="p2:AC1,R1,AL0"
pp2_trend=pp2_trend.rename(columns={"Category":"Method"})

pp3_trend=pd.merge(pp_category3,signup_trend3,how='left')
pp3_trend=pp3_trend[["date","name","email","phone_num","Category"]]
pp3_trend['category']="p3:AC1,R0,AL0"
pp3_trend=pp3_trend.rename(columns={"Category":"Method"})

pp4_trend=pd.merge(pp_category4,signup_trend3,how='left')
pp4_trend=pp4_trend[["date","name","email","phone_num","Category"]]
pp4_trend['category']="p4:AC0,R1,AL1"
pp4_trend=pp4_trend.rename(columns={"Category":"Method"})

pp5_trend=pd.merge(pp_category5,signup_trend3,how='left')
pp5_trend=pp5_trend[["date","name","email","phone_num","Category"]]
pp5_trend['category']="p5:AC0,R1,AL0"
pp5_trend=pp5_trend.rename(columns={"Category":"Method"})

paid_trend=pd.concat([pp1_trend,pp2_trend,pp3_trend,pp4_trend,pp5_trend],axis=0)

In [ ]:
#TO EXPORT- !!DATA HAS CHANGED !!

#drive.mount('/drive', force_remount = True)
#signup_trend3.to_csv('/drive/My Drive/signup_trend3.csv')
#np_category1.to_csv('/drive/My Drive/np_category1.csv')
#np_category2.to_csv('/drive/My Drive/np_category2.csv')
#np_category4.to_csv('/drive/My Drive/np_category4.csv')
#paid_all.to_csv('/drive/My Drive/paid_all.csv')
#paid_bankin.to_csv('/drive/My Drive/paid_bankin.csv')
#paid_cred2.to_csv('/drive/My Drive/paid_cred.csv')
#paid_free.to_csv('/drive/My Drive/paid_free.csv')
#p_category1.to_csv('/drive/My Drive/paid_category1.csv')
#p_category2.to_csv('/drive/My Drive/paid_category2.csv')
#p_category3.to_csv('/drive/My Drive/paid_category3.csv')
#p_category4.to_csv('/drive/My Drive/paid_category4.csv')

In [ ]:
# drive.mount('/drive', force_remount = True)
# signup_trend2.to_csv('/drive/My Drive/signup_trend.csv')
# signup_trend3.to_csv('/drive/My Drive/signup_trend3.csv')

In [ ]:
#NEW DATA TO EXPORT
#drive.mount('/drive', force_remount = True)
#np_category4.to_csv('/drive/My Drive/np_category4.csv')
#np_category5.to_csv('/drive/My Drive/np_category5.csv')
#pp_category4.to_csv('/drive/My Drive/p_category4.csv')
#pp_category5.to_csv('/drive/My Drive/p_category5.csv')
#paid_trend.to_csv('/drive/My Drive/paid_trend.csv')
#DONE

In [ ]:
#drive.mount('/drive', force_remount = True)
#free_reg_after.to_csv('/drive/My Drive/reg_after_13July.csv')

In [ ]:
paid_after=paid_after[paid_after["email"].isin(activecam_sub1.Email)]
print(len(paid_after))
paid_after=paid_after[~paid_after["email"].isin(paid_all.Email)]
paid_after = paid_after.drop_duplicates(subset="email", keep="last")  